In [5]:
import jax
import jax.numpy as jnp
from flax.struct import dataclass

In [10]:
@dataclass
class Gaussian:
    info: jnp.ndarray
    precision: jnp.ndarray
    dims: jnp.ndarray 

    @property
    def shape(self):
        return {
            "info": self.info.shape,
            "precision": self.precision.shape,
            "dims": self.dims.shape
        }
 
    @property
    def mean(self) -> jnp.ndarray:
        return jnp.linalg.inv(self.precision) @ self.info
    
    @property
    def covariance(self) -> jnp.ndarray:
        return jnp.linalg.inv(self.precision)
    
    @staticmethod
    def identity(variable: int) -> jnp.ndarray:
        dims = jnp.array([variable, variable, variable, variable])
        return Gaussian(jnp.zeros(4), jnp.eye(4), dims)
    
    def __call__(self, x: jnp.ndarray) -> float:
        return 1/2 * x.T @ self.precision @ x - self.info.T @ x

    def __getitem__(self, index) -> "Gaussian":
        return Gaussian(self.info[index], self.precision[index], self.dims[index])

    def __mul__(self, other: 'Gaussian') -> 'Gaussian':
        if other is None:
            return self.copy()
        
        dims = self.dims.copy()
        other_dims = other.dims.copy()
        other_unique_val = jnp.unique(other_dims, size=other_dims.shape[0]//4)[0]
        
        if dims.shape[0] == other_dims.shape[0] and (dims == other_dims).sum() == dims.shape[0]:
            idxs_self = idxs_other = jnp.arange(len(dims))
        if dims.shape[0] == 8:
            idxs_self = jnp.arange(len(dims), dtype=int)
            idxs_other = jnp.where(dims == other_unique_val, size=4)[0]
        else:
            idxs_self = jnp.arange(len(dims), dtype=int)
            idxs_other = jnp.arange(len(dims), len(dims) + len(other_dims)) 
            dims = jnp.concatenate((dims, other_dims))

        # Extend self matrix
        prec_self = jnp.zeros((len(dims), len(dims)))
        info_self = jnp.zeros((len(dims), 1))
        
        prec_self = prec_self.at[jnp.ix_(idxs_self, idxs_self)].set(self.precision)
        info_self = info_self.at[jnp.ix_(idxs_self, jnp.zeros(1, dtype=int))].set(self.info.reshape(len(self.dims), 1))
        # Extend other matrix
        prec_other = jnp.zeros((len(dims), len(dims)))
        info_other = jnp.zeros((len(dims), 1))
        
        prec_other = prec_other.at[jnp.ix_(idxs_other, idxs_other)].set(other.precision)
        info_other = info_other.at[jnp.ix_(idxs_other, jnp.zeros((1), dtype=int))].set(other.info.reshape(len(other.dims), 1))
        # Add
        prec = prec_other + prec_self
        info = info_other + info_self
        return Gaussian(info, prec, dims.astype(float))
    
    def marginalize(self, dims_to_remove) -> "Gaussian":
        info, prec = self.info.reshape(-1, 1), self.precision
        
        axis_a = jnp.where(self.dims != dims_to_remove[0], size=4)[0]
        axis_b = jax.lax.select(axis_a[0] == 0, jnp.arange(4, 8), jnp.arange(4))

        info_a = info[jnp.ix_(axis_a, jnp.zeros((1,), dtype=int))]
        prec_aa = prec[jnp.ix_(axis_a, axis_a)]
        info_b = info[jnp.ix_(axis_b, jnp.zeros((1, ), dtype=int))]
        prec_ab = prec[jnp.ix_(axis_a, axis_b)]
        prec_ba = prec[jnp.ix_(axis_b, axis_a)]
        prec_bb = prec[jnp.ix_(axis_b, axis_b)]

        diag_values = jnp.diag(prec_bb)
        prec_bb = jnp.fill_diagonal(prec_bb, jnp.where(diag_values == 0, 1, diag_values), inplace=False)
        prec_bb_inv = jnp.linalg.inv(prec_bb)
        # jax.debug.print("det: {}", jnp.linalg.det(prec_bb))
        # jax.debug.print("inv: {}", jnp.linalg.inv(prec_bb))
        info_ = info_a - prec_ab @ prec_bb_inv @ info_b
        prec_ = prec_aa - prec_ab @ prec_bb_inv @ prec_ba

        new_dims = jnp.full(4, self.dims[axis_a[0]])
        return Gaussian(info_.flatten(), prec_, new_dims)

In [11]:
same_g0 = Gaussian(jnp.ones(4,) * 1, jnp.eye(4) * 1, jnp.ones(4,) * 1)
same_g1 = Gaussian(jnp.ones(4,) * 1, jnp.eye(4) * 1, jnp.ones(4,) * 1)
same_g0 * same_g1

Gaussian(info=Array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32), precision=Array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32), dims=Array([1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32))

In [10]:
# same_g0 = Gaussian(jnp.ones(4,) * 1, jnp.eye(4) * 1, jnp.ones(4,) * 0)
# same_g1 = Gaussian(jnp.ones(4,) * 1, jnp.eye(4) * 1, jnp.ones(4,) * 1)
g0 = Gaussian(jnp.ones(4,) * 1, jnp.eye(4) * 1, jnp.ones((4,)) * 1)
g1 = Gaussian(jnp.ones(4,) * 2, jnp.eye(4) * 2, jnp.ones((4,)) * 2)
g0 * g1 

dims: [1. 1. 1. 1. 2. 2. 2. 2.]
idxs_self: [0 1 2 3]
idxs_other: [4 5 6 7]


Gaussian(info=Array([1., 1., 1., 1., 2., 2., 2., 2.], dtype=float32), precision=Array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2.]], dtype=float32), dims=Array([1., 1., 1., 1., 2., 2., 2., 2.], dtype=float32))

In [11]:
g0 = Gaussian(jnp.ones(4,) * 1, jnp.eye(4) * 1, jnp.ones(4,) * 1)
g1 = Gaussian(jnp.ones(4,) * 2, jnp.eye(4) * 2, jnp.ones(4,) * 2)
g2 = Gaussian(jnp.ones(4,) * 3, jnp.eye(4) * 3, jnp.ones(4,) * 3)
combined_gaussian = g0 * g1 * g2
combined_gaussian

dims: [1. 1. 1. 1. 2. 2. 2. 2.]
idxs_self: [0 1 2 3]
idxs_other: [4 5 6 7]
dims: [1. 1. 1. 1. 2. 2. 2. 2. 3. 3. 3. 3.]
idxs_self: [0 1 2 3 4 5 6 7]
idxs_other: [ 8  9 10 11]


Gaussian(info=Array([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.], dtype=float32), precision=Array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3.]], dtype=float32), dims=Array([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.], dtype=float32))

In [12]:
combined_gaussian * g2

dims: [1. 1. 1. 1. 2. 2. 2. 2. 3. 3. 3. 3.]
idxs_self: [ 0  1  2  3  4  5  6  7  8  9 10 11]
idxs_other: [ 8  9 10 11]


Gaussian(info=Array([1., 1., 1., 1., 2., 2., 2., 2., 6., 6., 6., 6.], dtype=float32), precision=Array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 6., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 6., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 6., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 6.]], dtype=float32), dims=Array([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.], dtype=float32))

In [13]:
combined_gaussian.marginalize(jnp.array([2.,2.,2.,2.]))

NonConcreteBooleanIndexError: Array boolean indices must be concrete; got ShapedArray(bool[12])

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.NonConcreteBooleanIndexError

In [ ]:
combined_gaussian.marginalize(jnp.array([3.,3.,3.,3.]))

axis a: [0 1 2 3 4 5 6 7]
axis b: [ 8  9 10 11]


Gaussian(info=Array([1., 1., 1., 1., 2., 2., 2., 2.], dtype=float32), precision=Array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2.]], dtype=float32), dims=Array([1., 1., 1., 1., 2., 2., 2., 2.], dtype=float32))